In [1]:
import os, sys
target = "AMP-Algo-Trading"
while os.path.basename(os.getcwd()) != target:
    os.chdir("..")
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())

In [2]:
from SQLite_Manager.sqlManager import SqlAlchemyDataBaseManager
import sqlite3
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")

In [3]:
directory = r"data/stock_market_data/sp500/csv"
tickers = r"data/tickers_list.csv"
path = r"backtesterClass"
dbName = "stockDB"

tickers_df = pd.read_csv(tickers)
ticker_list = set(tickers_df.iloc[:, 0].astype(str).str.upper())

db = SqlAlchemyDataBaseManager(f"{path}/{dbName}.db")
# Walk through the directory
for root, dirs, files in os.walk(directory):
    for i, file in enumerate(files):  # Use enumerate to track iteration count
        file_ticker = file[:-4].upper()  # Remove .csv and uppercase for comparison
        if file_ticker in ticker_list:
            try:
                df = pd.read_csv(os.path.join(directory, file))
                db.update(f"{file[:-4]}", df)
            except Exception as e:
                print(f"Error processing {file}: {e}")


In [ ]:
prin()


,Date,Close
count,10778,10778.000000
mean,2001-07-21 16:11:10.699573248,20.708647
min,1980-03-17 00:00:00,0.468750
25%,1990-11-09 18:00:00,11.750000
50%,2001-07-12 12:00:00,21.240000
75%,2012-03-28 18:00:00,27.889999
max,2022-12-12 00:00:00,54.889999
std,NaN,13.014009


In [4]:
db_path = r"backtesterClass/stockDB.db"  
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
# start_date, end_date = "01-01-2022", "12-12-2022"
# Get all table names (tickers)
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
tickers = [t[0] for t in tables]  


df_ini = pd.read_sql(f"SELECT Date, Close FROM {tickers[0]}", conn)
df_ini['Date'] = pd.to_datetime(df_ini['Date'], format="%d-%m-%Y")

for i,ticker in enumerate(tickers[1:]):
    try:
        df = pd.read_sql(f"SELECT Date, Close FROM {ticker}", conn)
        df['Date'] = pd.to_datetime(df['Date'], format="%d-%m-%Y")

        if i == 0:
            merged_df = pd.merge_asof(df_ini.sort_values('Date'), 
                                df.sort_values('Date'), 
                                on='Date', 
                                direction='backward',
                                suffixes=(f'_{tickers[0]}', f'_{ticker}')) 
        else:
            merged_df = pd.merge_asof(merged_df.sort_values('Date'), 
                                df.sort_values('Date'), 
                                on='Date', 
                                direction='backward',
                                suffixes=(f'_{tickers[i]}', f'_{ticker}'))
            
    except:
        print(ticker)
        pass

GS-PJ
BF-A
MS-PF
STZ-B


In [5]:
merged_df.dropna(axis=1)

,Date,Close_UAL,Close_ISRG,Close_NVR,Close_GILD,Close_EQIX,Close_SWKS,Close_MCHP,Close_CDNS,Close_LVS,...,Close_EFX,Close_SWK,Close_MU,Close_TRV,Close_AEP,Close_JNJ,Close_WM,Close_DOV,Close_WAT,Close
0,2006-02-06,33.900002,11.854444,748.250000,15.005000,46.480000,5.160000,18.530001,16.799999,48.430000,...,37.779999,48.930000,16.010000,44.130001,36.220001,56.849998,32.419998,31.003023,40.250000,23.730000
1,2006-02-07,32.990002,11.167778,739.849976,15.085000,46.689999,5.070000,18.385000,16.530001,46.700001,...,37.209999,48.299999,16.360001,43.770000,36.009998,56.799999,32.500000,30.299171,40.529999,23.670000
2,2006-02-08,32.000000,10.882222,752.000000,14.845000,48.000000,5.120000,18.344999,16.629999,46.889999,...,36.689999,48.610001,16.350000,43.680000,35.950001,58.599998,32.540001,30.346094,41.330002,23.990000
3,2006-02-09,34.500000,11.277778,735.000000,14.582500,47.250000,5.200000,18.049999,16.740000,47.380001,...,36.700001,48.849998,16.260000,43.459999,35.919998,58.549999,32.669998,30.641043,40.980000,23.719999
4,2006-02-10,34.240002,11.464444,732.000000,14.775000,47.500000,5.180000,18.065001,16.910000,47.709999,...,36.619999,49.099998,16.400000,43.630001,36.029999,58.389999,33.580002,30.587416,40.820000,23.969999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4238,2022-12-06,45.919998,266.309998,4600.000000,87.959999,686.869995,92.239998,75.139999,163.660004,46.799999,...,194.059998,77.730003,53.680000,188.500000,96.370003,176.100006,165.559998,139.770004,337.589996,12.750000
4239,2022-12-07,44.060001,267.959991,4725.000000,87.790001,685.119995,91.959999,75.230003,160.589996,47.130001,...,193.839996,77.250000,53.820000,188.050003,96.669998,177.169998,165.210007,138.350006,340.640015,12.810000
4240,2022-12-08,42.799999,272.019989,4694.279785,89.470001,683.340027,95.169998,76.820000,164.059998,48.310001,...,198.080002,78.269997,55.200001,187.720001,97.720001,177.199997,166.009995,137.259995,343.040009,13.300000
4241,2022-12-09,43.040001,270.600006,4648.529785,87.970001,675.799988,94.519997,75.879997,163.669998,47.310001,...,197.449997,79.230003,54.869999,185.649994,96.580002,175.740005,166.830002,137.289993,336.529999,13.440000


,Date,Close_UAL,Close_ISRG,Close_NVR,Close_GILD,Close_NLSN,Close_EQIX,Close_V,Close_QRVO,Close_FLT,...,Close_EFX,Close_SWK,Close_MU,Close_TRV,Close_AEP,Close_JNJ,Close_WM,Close_DOV,Close_WAT,Close
count,4243,4243.000000,4243.000000,4243.000000,4243.000000,2949.000000,4243.000000,3711.000000,2001.000000,3019.000000,...,4243.000000,4243.000000,4243.000000,4243.000000,4243.000000,4243.000000,4243.000000,4243.000000,4243.000000,4243.000000
mean,2014-07-10 07:32:44.176290560,42.471737,96.374251,1848.518686,53.170215,32.675242,301.585058,92.309700,92.342119,162.196356,...,92.467697,97.396222,27.168936,93.759972,57.560157,101.347782,65.735925,64.859927,139.416743,25.416396
min,2006-02-06 00:00:00,3.130000,9.481111,319.559998,13.437500,11.850000,35.139999,10.605000,34.529999,25.250000,...,19.790001,22.750000,1.690000,30.500000,24.280001,46.599998,22.230000,14.787604,30.750000,4.540000
25%,2010-04-24 12:00:00,23.835000,35.136667,683.279999,22.670000,24.990000,92.494999,23.705000,65.930000,113.404999,...,36.880001,58.055000,8.580000,52.775000,38.965000,64.570000,34.900002,33.912281,69.145000,17.260000
50%,2014-07-11 00:00:00,40.910000,56.259998,1150.250000,62.790001,30.160000,215.210007,70.000000,77.099998,154.029999,...,72.750000,86.129997,16.719999,92.919998,52.880001,98.260002,44.840000,54.733440,106.209999,23.309999
75%,2018-09-25 12:00:00,57.299999,163.903336,2961.094971,73.639999,42.810001,444.510010,155.630005,111.800003,228.240005,...,131.655006,134.585007,42.834999,127.324997,74.014999,133.200005,89.924999,82.722134,195.700005,30.275001
max,2022-12-12 00:00:00,96.699997,365.420013,5959.330078,122.209999,55.810001,882.830017,250.929993,199.539993,328.850006,...,297.049988,220.690002,97.360001,189.809998,105.180000,186.009995,175.289993,182.919998,424.700012,54.889999
std,NaN,22.788206,87.897019,1456.633334,29.058013,10.731354,233.841088,72.082702,40.427147,80.661394,...,63.338856,45.612943,23.092596,40.898443,20.920494,38.350599,40.196593,38.403991,88.316944,10.842153
